In [126]:
from markdown import markdown

with open('README_AC.md', 'r') as file:
    contenido_md = file.read()

In [127]:
contenido_md

'# Bienvenido a la comunidad de Inner Source - Welcome to the Inner Source community\n\n![IS TEC](imgs/TEC_logo.png)\n\nSi has llegado hasta aquÃ\xad, dejame que te de la bienvenida a nuestra comunidad de Inner Source para MAPFRE. Nos complace mucho tu interÃ©s en nuestra comunidad y te animamos a que, tÃº y tu equipo, seais una parte importante de la comunidad aportando vuestra colaboraciÃ³n.\n\n1. Resumen del proyecto - Project Overview\n\n    Â¿QuÃ© queremos hacer en la comunidad de Inner Source?\n    Nuestra principal meta, es apoyaros y compaÃ±aros en todo lo necesario para que vuestros proyectos dispongan de las herramientas y de la metodologÃ\xada de MAPFRE, de una manera Ã¡gil y rÃ¡pida. **Queremos ser un equipo "facilitador" dentro de la compaÃ±Ã\xada.** Queremos hacer disponible una serie de herramientas que permitan gestionar los diferentes repositorios de la compaÃ±ia de manera que quien quiera o lo necesite pueda colaborar en el desarrollo de las diversas soluciones que es

In [65]:
import nltk
from nltk import ne_chunk, pos_tag

def extract_person_entities(tokens):
    tagged_tokens = pos_tag(tokens)
    tree = ne_chunk(tagged_tokens)
    
    person_entities = []
    for subtree in tree:
        if isinstance(subtree, nltk.Tree) and subtree.label() == 'PERSON':
            entity = " ".join([token for token, pos in subtree.leaves()])
            person_entities.append(entity)

    non_person_tokens = [token for token, pos in tagged_tokens if token not in person_entities]

    return non_person_tokens


In [130]:
import re
import nltk
import spacy
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import CountVectorizer
from langdetect import detect_langs

# Descargar recursos necesarios
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('wordnet')

# Cargar modelos de spaCy para inglés y español
nlp_en = spacy.load("en_core_web_sm")
nlp_es = spacy.load("es_core_news_sm")

# Preprocesamiento básico
def preprocesing(text):
    # Eliminar palabras que estén completamente en mayúsculas
    text = ' '.join(word for word in text.split() if not word.isupper())
    # Eliminar dígitos
    text = re.sub(r'\d+', '', text)
    # Tokenizar el texto
    text = nltk.word_tokenize(text)
    # EXTRACCION ENTITIES 
    text = extract_person_entities(text)

    return text

tokens = preprocesing(contenido_md)

# Lista de idiomas y sus códigos para stopwords
stopwords_lang = {
    "spanish": set(stopwords.words("spanish")),
    "english": set(stopwords.words("english")),
    "comun": {"actual", "central", "normal", "base", "final", "local", "son", "simple",
              "popular", "radio", "total", "color", "describe", "sin", "sea", "use", "oral",
              "error", "familiar", "global", "informal", "original", "social", "super", "universal",
              "capital", "continental", "comun", "festival", "final", "ideal", "legal", "natural",
              "personal", "real", "tradicional", "virtual", "digital", "material", "mental",
              "social", "visual", "editor", "horizontal", "vertical"}
}

# Eliminar stopwords para cada idioma
tokens = [token.lower() for token in tokens if token.isalnum()]
tokens = [token for token in tokens if token not in stopwords_lang["spanish"] and 
          token not in stopwords_lang["english"] and token not in stopwords_lang["comun"]]

# Funciones de idioma usando WordNet
def is_english_word(word):
    synsets = wordnet.synsets(word)
    return len(synsets) > 0

def is_spanish_word(word):
    synsets = wordnet.synsets(word, lang='spa')
    return bool(synsets)


# Calcular probabilidades de pertenencia a cada idioma y clasificar palabras
def classify_words(tokens):
    english_words = set()
    spanish_words = set()

    for token in tokens:
        english = is_english_word(token)
        spanish = is_spanish_word(token)

        if english and not spanish:
            english_words.add(token)
        elif spanish and not english:
            spanish_words.add(token)
        else:
            # Si la palabra es ambigua, utiliza langdetect
            try:
                detected_langs = detect_langs(token)
                detected_lang = max(detected_langs, key=lambda x: x.prob).lang
                if detected_lang == 'en':
                    english_words.add(token)
                else: spanish_words.add(token)
            
            except:
                continue

    return english_words, spanish_words

# Clasificar palabras
english_words, spanish_words = classify_words(tokens)

# Lematizar palabras clasificadas por idioma
english_lemmatized = list(set([nlp_en(word)[0].lemma_ for word in english_words]))
spanish_lemmatized = list(set([nlp_es(word)[0].lemma_ for word in spanish_words]))

# Mostrar los resultados
print("English Words:", english_lemmatized)
print("Spanish Words:", spanish_lemmatized)

# Calcular y mostrar los porcentajes
total_words = len(english_lemmatized) + len(spanish_lemmatized)
english_percentage = (len(english_lemmatized) / total_words) * 100 if total_words > 0 else 0
spanish_percentage = (len(spanish_lemmatized) / total_words) * 100 if total_words > 0 else 0

print(f"English: {english_percentage:.2f}%")
print(f"Spanish: {spanish_percentage:.2f}%")


English Words: ['toda', 'groovy', 'trust', 'currently', 'way', 'allow', 'convention', 'definition', 'project', 'automation', 'different', 'poseidon', 'current', 'code', 'conduct', 'example', 'good', 'tarea', 'implementation', 'accompany', 'apply', 'main', 'support', 'methodology', 'moment', 'check', 'document', 'collaborate', 'team', 'story', 'work', 'fork', 'series', 'part', 'pillar', 'inner', 'overview', 'documentation', 'development', 'agreement', 'whoever', 'victor', 'want', 'contact', 'glance', 'read', 'phrase', 'available', 'owner', 'look', 'simple', 'member', 'see', 'contribute', 'information', 'encourage', 'task', 'section', 'generate', 'exist', 'we', 'product', 'take', 'repository', 'message', 'commit', 'modality', 'root', 'role', 'recommend', 'configure', 'abstract', 'focus', 'fast', 'user', 'facilitator', 'publication', 'update', 'welcome', 'within', 'automatism', 'practice', 'begin', 'complex', 'tool', 'regularly', 'everything', 'need', 'manage', 'solution', 'account', 'val

In [125]:
print(total_words)
print(len(english_lemmatized))
print(len(spanish_lemmatized))

98
94
4
